In [1]:
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from tabulate import tabulate

import warnings

warnings.filterwarnings("ignore")

In [2]:
import os

input_dir = "./kaggle/input"
for dirname, _, filenames in os.walk(input_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv


In [3]:
loan_original = pd.read_csv(
    os.path.join(
        input_dir, "loan-approval-prediction-dataset/loan_approval_dataset.csv"
    )
)
# columns have weird spaces in names:
loan_original.columns = loan_original.columns.str.replace(" ", "")
loan_original[loan_original["loan_id"].duplicated(keep=False) == True].sort_values(
    ["loan_id"]
)
# replace all spaces in loan_status and education columns to
loan_original["loan_status"] = loan_original["loan_status"].str.replace(" ", "")
loan_original["education"] = loan_original["education"].str.replace(" ", "")

# remove the loan_id column
loan_ds = loan_original.drop(["loan_id"], axis=1)

GRADUATE_STRING = loan_ds["education"].unique()[0]
NOT_GRADUATE_STRING = loan_ds["education"].unique()[1]

APPROVED_STRING = loan_ds["loan_status"].unique()[0]
REJECTED_STRING = loan_ds["loan_status"].unique()[1]


print(loan_original.columns)

print(
    f"{GRADUATE_STRING=}, {NOT_GRADUATE_STRING=}, {APPROVED_STRING=}, {REJECTED_STRING=}"
)

# print how many rows and columns are in the dataset
original_num_rows, original_num_columns = loan_original.shape
loan_original.sample(7)

Index(['loan_id', 'no_of_dependents', 'education', 'self_employed',
       'income_annum', 'loan_amount', 'loan_term', 'cibil_score',
       'residential_assets_value', 'commercial_assets_value',
       'luxury_assets_value', 'bank_asset_value', 'loan_status'],
      dtype='object')
GRADUATE_STRING='Graduate', NOT_GRADUATE_STRING='NotGraduate', APPROVED_STRING='Approved', REJECTED_STRING='Rejected'


,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
3330,3331,1,Graduate,No,6300000,17200000,12,714,10100000,4200000,24700000,8500000,Approved
2190,2191,2,Graduate,Yes,5600000,21000000,18,883,9800000,1600000,20900000,6000000,Approved
847,848,2,Graduate,No,7700000,15400000,18,524,12900000,5100000,21000000,6600000,Rejected
4130,4131,0,NotGraduate,Yes,6200000,14900000,8,855,-100000,7200000,14200000,8200000,Approved
2681,2682,4,NotGraduate,No,1500000,3100000,6,885,100000,2800000,3800000,1900000,Approved
1529,1530,3,Graduate,Yes,9000000,33600000,2,426,21600000,16200000,20700000,11900000,Approved
2785,2786,0,NotGraduate,No,7700000,29300000,14,801,6200000,11800000,23400000,4700000,Approved
